# Pytorch图像分类模型转ONNX-ImageNet1000类

把Pytorch预训练ImageNet图像分类模型，导出为ONNX格式，用于后续在推理引擎上部署。

代码运行云GPU平台：公众号 人工智能小技巧 回复 gpu

同济子豪兄 2022-8-22 2023-4-28 2023-5-8

## 导入工具包

In [1]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 载入ImageNet预训练PyTorch图像分类模型

In [2]:
model = models.resnet18(pretrained=True)
model = model.eval().to(device)

d:\Program\Anaconda\envs\pytorch_GPU\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program\Anaconda\envs\pytorch_GPU\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\unicorn/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 24.7MB/s]


## 构造一个输入图像Tensor

In [3]:
x = torch.randn(1, 3, 256, 256).to(device)

## 输入Pytorch模型推理预测，获得1000个类别的预测结果

In [4]:
output = model(x)

In [5]:
output.shape

torch.Size([1, 1000])

## Pytorch模型转ONNX格式

In [6]:
with torch.no_grad():
    torch.onnx.export(
        model,                       # 要转换的模型
        x,                           # 模型的任意一组输入
        'resnet18_imagenet.onnx',    # 导出的 ONNX 文件名
        opset_version=11,            # ONNX 算子集版本
        input_names=['input'],       # 输入 Tensor 的名称（自己起名字）
        output_names=['output']      # 输出 Tensor 的名称（自己起名字）
    ) 

## 验证onnx模型导出成功

In [7]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('resnet18_imagenet.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错, onnx模型载入成功')

无报错，onnx模型载入成功


## 以可读的形式打印计算图

In [8]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph main_graph (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %fc.weight[FLOAT, 1000x512]
  %fc.bias[FLOAT, 1000]
  %onnx::Conv_193[FLOAT, 64x3x7x7]
  %onnx::Conv_194[FLOAT, 64]
  %onnx::Conv_196[FLOAT, 64x64x3x3]
  %onnx::Conv_197[FLOAT, 64]
  %onnx::Conv_199[FLOAT, 64x64x3x3]
  %onnx::Conv_200[FLOAT, 64]
  %onnx::Conv_202[FLOAT, 64x64x3x3]
  %onnx::Conv_203[FLOAT, 64]
  %onnx::Conv_205[FLOAT, 64x64x3x3]
  %onnx::Conv_206[FLOAT, 64]
  %onnx::Conv_208[FLOAT, 128x64x3x3]
  %onnx::Conv_209[FLOAT, 128]
  %onnx::Conv_211[FLOAT, 128x128x3x3]
  %onnx::Conv_212[FLOAT, 128]
  %onnx::Conv_214[FLOAT, 128x64x1x1]
  %onnx::Conv_215[FLOAT, 128]
  %onnx::Conv_217[FLOAT, 128x128x3x3]
  %onnx::Conv_218[FLOAT, 128]
  %onnx::Conv_220[FLOAT, 128x128x3x3]
  %onnx::Conv_221[FLOAT, 128]
  %onnx::Conv_223[FLOAT, 256x128x3x3]
  %onnx::Conv_224[FLOAT, 256]
  %onnx::Conv_226[FLOAT, 256x256x3x3]
  %onnx::Conv_227[FLOAT, 256]
  %onnx::Conv_229[FLOAT, 256x128x1x1]
  %onnx::Conv_230[FLOAT, 256]
  %onnx::Conv_2

## 使用Netron可视化模型结构

Netron：https://netron.app

视频教程：https://www.bilibili.com/video/BV1TV4y1P7AP